Pythonがローカルにインストールされていないパソコンでも、TryJupyterのオンラインのページで動くように作ってあります。

https://mybinder.org/v2/gh/ipython/ipython-in-depth/master?filepath=binder/Index.ipynb

最初のセルでxlrdを入れないと動かなかったので、最初のセルを置いてあります。

In [1]:
!pip install xlrd

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 4)
pd.set_option("display.max_columns", 4)

In [3]:
input_book = pd.ExcelFile('Int_papers_scopus2016.xlsx')  
input_sheet_name = input_book.sheet_names
df = input_book.parse(input_sheet_name[0])

In [4]:
#Affiliationsが；で区切られているのでそこでsplitする 
#pd.concat()で連結（結合）するときに必要な列（Affiliations）だけ選択

df3 = pd.concat([df['Affiliations'].str.split(';', expand=True)], axis=1)

print('論文数は')
print(len(df3))

print('最も多い機関数は')
print(len(df3.columns))

論文数は
604
最も多い機関数は
89


In [5]:
#行と列を入れ替えたデータフレーム
df4 = df3.T

df10 = pd.DataFrame(df4.count())
#カラムに名前を付ける
df10.columns = ['Affiliations']

#各論文（各カラム）の中に'Japan’所属の機関の数がいくつあるか
#Japanの機関数をdf5とする
#python pandas forループの計算結果をデータフレームに入れる方法がわからなくて検索したところ、
#一度numpy配列に入れてからデータフレームに変換することでできるとの方法を見つけてやってみる　→できた

x = np.zeros(len(df4.columns), dtype=np.int64)  #データ型をintに指定
for i in range(len(df4.columns)):
        df5 = df4[i].str.contains('Japan').sum()
        x[i] = df5
df5 = pd.DataFrame(x)
df5.columns = ['Japan']

#各論文（各カラム）の機関数と比較して、'Japan'所属の機関数と等しければ国内論文、数が異なれば国際共著論文と判断する

#各論文の機関の数（df10)と日本機関の数(df5)を一つのデータフレームにする
df11 = pd.concat([df10, df5], axis=1)

#機関の数と日本機関の数が等しくないところをTrue=Internationalとする
df11['International'] = (df11['Affiliations']!=df11['Japan'])

print('この年のscopus論文の総数は')
print(len(df4.columns))

International = df11['International']==True
print('国際共著論文は')
print(International.sum())

Domestic = df11['International']==False
print('日本のみの論文数は')
print(Domestic.sum())

print('国際共著論文の比率は')
print(str(round(International.sum()/len(df4.columns)*100,2)) +'%')

この年のscopus論文の総数は
604
国際共著論文は
106
日本のみの論文数は
498
国際共著論文の比率は
17.55%
